In [23]:
import pandas as pd
import numpy as np
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.arima_process import arma_generate_sample
from sklearn.metrics import mean_squared_error
from math import sqrt

In [24]:
df = pd.read_csv('https://raw.githubusercontent.com/thieu1995/csv-files/main/data/timeseries/univariate/AirPassengers.csv')
df

,Month,#Passengers
0,1949-01,112
1,1949-02,118
2,1949-03,132
3,1949-04,129
4,1949-05,121
...,...,...
139,1960-08,606
140,1960-09,508
141,1960-10,461
142,1960-11,390


In [25]:
# Chia dữ liệu thành tập huấn luyện và kiểm tra
train_data = df[:int(len(df)*0.7)]
test_data = df[int(len(df)*0.7):]

In [26]:
best_aic = np.inf 
best_order = None
best_mdl = None

pq_rng = range(5) # [0,1,2,3,4]
d_rng = range(2) # [0,1]
for i in pq_rng:
    for d in d_rng:
        for j in pq_rng:
            try:
                tmp_mdl = ARIMA(train_data.values, order=(i,d,j)).fit(method='mle', trend='nc')
                tmp_aic = tmp_mdl.aic
                if tmp_aic < best_aic:
                    best_aic = tmp_aic
                    best_order = (i, d, j)
                    best_mdl = tmp_mdl
            except: continue


print('aic: {:6.5f} | order: {}'.format(best_aic, best_order))


aic:    inf | order: None


In [27]:
models = [AutoReg, ARIMA]
for model in models:
    model = model(train_data, lags=1).fit()
    predictions = model.predict(start=len(train_data), end=len(train_data) + len(test_data)-1, dynamic=False)
    rmse = sqrt(mean_squared_error(test_data, predictions))
    print(f'RMSE for {model.__class__.__name__}: ', rmse)

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).